In [ ]:
import os
import pickle
import math
import numpy as np
import imageio
import matplotlib.pyplot as plt

from implicitpleth.utils.algorithmic import CHROME_DEHAAN, POS_WANG, detrend_and_filter, ICA_POH, PBV
from implicitpleth.utils.eval import eval_performance, eval_clinical_performance, eval_performance_bias
from implicitpleth.utils.errors import getErrors
from implicitpleth.utils.utils import prpsd2

# 330 Window, 128 Stride

In [ ]:
PPG_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/rgb_files/'
NPY_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/'
_trial = 'v_63_2'

session_names = os.listdir(PPG_ROOT)

error_green_list = []
error_avg_list = []
error_chrom_list = []
error_pos_list = []
error_ica_list = []
error_pbv_list = []
error_green_proc_list = []
green_list = []
avg_list = []
chrom_list = []
pos_list = []
ica_list = []
pbv_list = []
gt_list = []
green_proc_list = []
for _trial in session_names:
    print(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg_full = np.load(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))[25:]

    print(os.path.join(NPY_ROOT, f'residual_0_{_trial}.npy'))
    pleth_1 = np.load(os.path.join(NPY_ROOT, f'residual_0_{_trial}.npy')).mean(1).mean(1)
    pleth_1 = pleth_1 - np.mean(pleth_1, axis=0, keepdims=True)
    pleth_2 = np.load(os.path.join(NPY_ROOT, f'residual_300_{_trial}.npy')).mean(1).mean(1)
    pleth_2 = pleth_2 - np.mean(pleth_2, axis=0, keepdims=True)
    pleth_3 = np.load(os.path.join(NPY_ROOT, f'residual_600_{_trial}.npy')).mean(1).mean(1)
    pleth_3 = pleth_3 - np.mean(pleth_3, axis=0, keepdims=True)
    pleth_full = np.concatenate((pleth_1, pleth_2, pleth_3), axis=0)
    temp_green = []
    temp_avg = []
    temp_chrom = []
    temp_pos = []
    temp_ica = []
    temp_pbv = []
    temp_gt = []
    temp_proc = []
    for i in [0,128,256,384,512]:
        gt_ppg = gt_ppg_full[i:i+300]
        gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'GT: {gt_hr}')
        temp_gt.append(gt_hr)
        
        pleth = pleth_full[i:i+300]
        green = pleth[:,1]
        green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
        green_proc = detrend_and_filter(pleth[:,1])
        green_proc_hr = prpsd2(green_proc-np.mean(green_proc), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Proc Est: {green_proc_hr}    ;   Green Proc Error: {np.abs(green_proc_hr - gt_hr)}')
        avg = pleth.mean(1)
        avg_hr = prpsd2(avg-np.mean(avg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Avg Est: {avg_hr}    ;   Avg Error: {np.abs(avg_hr - gt_hr)}')
        chrome_dehaan = CHROME_DEHAAN(pleth)
        chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
        pos_wang = POS_WANG(pleth)
        pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
        ica_poh = ICA_POH(pleth)
        ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')
        pbv_est = PBV(pleth)
        pbv_hr = prpsd2(pbv_est-np.mean(pbv_est), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'PBV Est: {pbv_hr}    ;   PBV Error: {np.abs(pbv_hr - gt_hr)}')
        
        temp_green.append(green_hr)
        temp_avg.append(avg_hr)
        temp_chrom.append(chrome_dehaan_hr)
        temp_pos.append(pos_wang_hr)
        temp_ica.append(ica_hr)
        temp_pbv.append(pbv_hr)
        temp_proc.append(green_proc_hr)
        print('*'*100)
    hr_gt_windowed = np.array(temp_gt)

    hr_est_windowed = np.array([temp_green])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_list.append(MAE)
    
    hr_est_windowed = np.array([temp_avg])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_avg_list.append(MAE)
    
    hr_est_windowed = np.array([temp_pos])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_pos_list.append(MAE)
    
    hr_est_windowed = np.array([temp_chrom])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_chrom_list.append(MAE)
    
    hr_est_windowed = np.array([temp_ica])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_ica_list.append(MAE)

    hr_est_windowed = np.array([temp_pbv])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_pbv_list.append(MAE)

    hr_est_windowed = np.array([temp_proc])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_proc_list.append(MAE)
    
    green_list.append(temp_green)
    avg_list.append(temp_avg)
    chrom_list.append(temp_chrom)
    pos_list.append(temp_pos)
    ica_list.append(temp_ica)
    pbv_list.append(temp_pbv)
    green_proc_list.append(temp_proc)
    
    gt_list.append(temp_gt)

    # Errors

    print('-'*100)

In [ ]:
print(f'green_error_list: {np.mean(error_green_list)}')
print(f'proc_error_list: {np.mean(error_green_proc_list)}')
print(f'pos_error_list: {np.mean(error_pos_list)}')
print(f'chrom_error_list: {np.mean(error_chrom_list)}')
print(f'ica_error_list: {np.mean(error_ica_list)}')
print(f'pbv_error_list: {np.mean(error_pbv_list)}')
print(f'avg_error_list: {np.mean(error_avg_list)}')

In [ ]:
fitz_labels_path='/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/fitzpatrick_labels.pkl'
print(100*"-")
print('!!   !!  Green  !!  !!')
eval_clinical_performance(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  proc  !!  !!')
eval_clinical_performance(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  pos  !!  !!')
eval_clinical_performance(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  chrom  !!  !!')
eval_clinical_performance(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  ica  !!  !!')
eval_clinical_performance(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  pbv  !!  !!')
eval_clinical_performance(hr_est=np.array(pbv_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(pbv_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  avg  !!  !!')
eval_clinical_performance(hr_est=np.array(avg_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(avg_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

In [ ]:
_trial = 'v_6_1'
# _trial = 'v_36_2'
# _trial = 'v_45_3'
# _trial = 'v_101_1'
# _trial = 'v_58_3'
# _trial = 'v_66_1'
# _trial = 'v_67_5'
pleth_1 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_0_{_trial}.npy').mean(1).mean(1)
pleth_1 = pleth_1 - np.mean(pleth_1, axis=0, keepdims=True)
pleth_2 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_300_{_trial}.npy').mean(1).mean(1)
pleth_2 = pleth_2 - np.mean(pleth_2, axis=0, keepdims=True)
pleth_3 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_600_{_trial}.npy').mean(1).mean(1)
pleth_3 = pleth_3 - np.mean(pleth_3, axis=0, keepdims=True)
pleth_full = np.concatenate((pleth_1, pleth_2, pleth_3), axis=0)
plt.figure(figsize=(30,10))
plt.plot(pleth_full[:,1])

# Baselines

In [ ]:
ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/rgb_files/'

session_names = os.listdir(ROOT)

error_green_list = []
error_avg_list = []
error_chrom_list = []
error_pos_list = []
error_ica_list = []
error_pbv_list = []
error_green_proc_list = []
green_list = []
avg_list = []
chrom_list = []
pos_list = []
ica_list = []
pbv_list = []
gt_list = []
green_proc_list = []
for _trial in session_names:
    print(os.path.join(ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg_full = np.load(os.path.join(ROOT,_trial,"rgbd_ppg.npy"))[25:]

    pleth_full  = []
    for idx in range(900):
        pleth_full.append(imageio.v2.imread(os.path.join(ROOT, _trial, f'rgbd_rgb_{idx}.png')))
    pleth_full = np.array(pleth_full).mean(1).mean(1) / 255
    
    temp_green = []
    temp_avg = []
    temp_chrom = []
    temp_pos = []
    temp_ica = []
    temp_pbv = []
    temp_gt = []
    temp_proc = []
    for i in [0,128,256,384,512]:
        gt_ppg = gt_ppg_full[i:i+300]
        gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'GT: {gt_hr}')
        temp_gt.append(gt_hr)
        
        pleth = pleth_full[i:i+300]
        green = pleth[:,1]
        green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
        green_proc = detrend_and_filter(pleth[:,1])
        green_proc_hr = prpsd2(green_proc-np.mean(green_proc), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Proc Est: {green_proc_hr}    ;   Green Proc Error: {np.abs(green_proc_hr - gt_hr)}')
        avg = pleth.mean(1)
        avg_hr = prpsd2(avg-np.mean(avg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Avg Est: {avg_hr}    ;   Avg Error: {np.abs(avg_hr - gt_hr)}')
        chrome_dehaan = CHROME_DEHAAN(pleth)
        chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
        pos_wang = POS_WANG(pleth)
        pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
        ica_poh = ICA_POH(pleth)
        ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')
        pbv_est = PBV(pleth)
        pbv_hr = prpsd2(pbv_est-np.mean(pbv_est), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'PBV Est: {pbv_hr}    ;   PBV Error: {np.abs(pbv_hr - gt_hr)}')
        
        temp_green.append(green_hr)
        temp_avg.append(avg_hr)
        temp_chrom.append(chrome_dehaan_hr)
        temp_pos.append(pos_wang_hr)
        temp_ica.append(ica_hr)
        temp_pbv.append(pbv_hr)
        temp_proc.append(green_proc_hr)
        print('*'*100)
    hr_gt_windowed = np.array(temp_gt)

    hr_est_windowed = np.array([temp_green])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_list.append(MAE)
    
    hr_est_windowed = np.array([temp_avg])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_avg_list.append(MAE)
    
    hr_est_windowed = np.array([temp_pos])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_pos_list.append(MAE)
    
    hr_est_windowed = np.array([temp_chrom])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_chrom_list.append(MAE)
    
    hr_est_windowed = np.array([temp_ica])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_ica_list.append(MAE)

    hr_est_windowed = np.array([temp_pbv])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_pbv_list.append(MAE)

    hr_est_windowed = np.array([temp_proc])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_proc_list.append(MAE)
    
    green_list.append(temp_green)
    avg_list.append(temp_avg)
    chrom_list.append(temp_chrom)
    pos_list.append(temp_pos)
    ica_list.append(temp_ica)
    pbv_list.append(temp_pbv)
    green_proc_list.append(temp_proc)
    
    gt_list.append(temp_gt)

    # Errors

    print('-'*100)

In [ ]:
print(f'green_error_list: {np.mean(error_green_list)}')
print(f'proc_error_list: {np.mean(error_green_proc_list)}')
print(f'pos_error_list: {np.mean(error_pos_list)}')
print(f'chrom_error_list: {np.mean(error_chrom_list)}')
print(f'ica_error_list: {np.mean(error_ica_list)}')
print(f'pbv_error_list: {np.mean(error_pbv_list)}')
print(f'avg_error_list: {np.mean(error_avg_list)}')

In [ ]:
fitz_labels_path='/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/fitzpatrick_labels.pkl'
print(100*"-")
print('!!   !!  Green  !!  !!')
eval_clinical_performance(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  proc  !!  !!')
eval_clinical_performance(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  pos  !!  !!')
eval_clinical_performance(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  chrom  !!  !!')
eval_clinical_performance(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  ica  !!  !!')
eval_clinical_performance(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  pbv  !!  !!')
eval_clinical_performance(hr_est=np.array(pbv_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(pbv_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  avg  !!  !!')
eval_clinical_performance(hr_est=np.array(avg_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(avg_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

# Single Files

In [ ]:
# _trial = 'v_6_2'
# _trial = 'v_7_2'
# _trial = 'v_36_2'
# _trial = 'v_45_3'
# _trial = 'v_100_1'
# _trial = 'v_58_3'
# _trial = 'v_66_1'   
# _trial = 'v_67_5'
_trial = 'v_30_6'
pleth_1 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_0_{_trial}.npy').mean(1).mean(1)
pleth_1 = pleth_1 - np.mean(pleth_1, axis=0, keepdims=True)
pleth_2 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_300_{_trial}.npy').mean(1).mean(1)
pleth_2 = pleth_2 - np.mean(pleth_2, axis=0, keepdims=True)
pleth_3 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_600_{_trial}.npy').mean(1).mean(1)
pleth_3 = pleth_3 - np.mean(pleth_3, axis=0, keepdims=True)
pleth_full = np.concatenate((pleth_1, pleth_2, pleth_3), axis=0)[:300,1]
pleth_res_hr = prpsd2(pleth_full-np.mean(pleth_full), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
plt.figure(figsize=(30,3))
plt.plot(-pleth_full)
plt.show()

pleth_full  = []
for idx in range(900):
    pleth_full.append(imageio.v2.imread(os.path.join('/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/rgb_files/', _trial, f'rgbd_rgb_{idx}.png')))
pleth_full = np.array(pleth_full).mean(1).mean(1) / 255
pleth_full = POS_WANG(pleth_full[:300])
pleth_base_hr = prpsd2(pleth_full-np.mean(pleth_full), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
plt.figure(figsize=(30,3))
plt.plot(-pleth_full)
# plt.plot(-pleth_full[:,1])
plt.show()

gt_ppg_full = np.load(os.path.join('/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/rgb_files/',_trial,"rgbd_ppg.npy"))[25:325]
gt_hr = prpsd2(gt_ppg_full-np.mean(gt_ppg_full), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
print(_trial)
print("GT:", gt_hr)
print("Res:", pleth_res_hr)
print("Base:", pleth_base_hr)

# Save Waveforms

In [ ]:
# ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/rgb_files/'
# SAVE_PATH = '/home/pradyumnachari/Desktop/WaveformsNpy'
# session_names = os.listdir(ROOT)

# for _trial in session_names: 
#     pleth_1 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_0_{_trial}.npy').mean(1).mean(1)
#     pleth_2 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_300_{_trial}.npy').mean(1).mean(1)
#     pleth_3 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_600_{_trial}.npy').mean(1).mean(1)
#     pleth_full = np.concatenate((pleth_1, pleth_2, pleth_3), axis=0)
#     green_pleth = pleth_full[:,1]
#     np.save(os.path.join(SAVE_PATH, f'res_green_{_trial}.npy'), green_pleth)
#     ica_pleth = ICA_POH(pleth_full)
#     np.save(os.path.join(SAVE_PATH, f'res_ica_{_trial}.npy'), ica_pleth)

#     pleth_full  = []
#     for idx in range(900):
#         pleth_full.append(imageio.v2.imread(os.path.join('/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/rgb_files/', _trial, f'rgbd_rgb_{idx}.png')))
#     pleth_full = np.array(pleth_full).mean(1).mean(1) / 255
#     green_pleth = pleth_full[:,1]
#     np.save(os.path.join(SAVE_PATH, f'orig_green_{_trial}.npy'), green_pleth)
#     chrom_pleth = CHROME_DEHAAN(pleth_full)
#     np.save(os.path.join(SAVE_PATH, f'orig_chrom_{_trial}.npy'), chrom_pleth)
#     pos_pleth = POS_WANG(pleth_full)
#     np.save(os.path.join(SAVE_PATH, f'orig_pos_{_trial}.npy'), pos_pleth)
#     ica_pleth = ICA_POH(pleth_full)
#     np.save(os.path.join(SAVE_PATH, f'orig_ica_{_trial}.npy'), ica_pleth)